# ___

# [ Machine Learning in Geosciences ]

**Department of Applied Geoinformatics and Carthography, Charles University** 

*Lukas Brodsky lukas.brodsky@natur.cuni.cz*

    
___

# Single Layer Neural Network 

The purpose of this notebook is to  demonstrate a single layer non-linear neural network. 

In [ ]:
import torch
import torch.nn as nn  
import matplotlib.pyplot as plt

In [ ]:
# synthetic the data (multi-step function / non-linear) 
X = torch.arange(-30, 30, 1).view(-1, 1).type(torch.FloatTensor)
X.shape

In [ ]:
Y = torch.zeros(X.shape[0])
Y[(X[:, 0] <= -10)] = 1.0
Y[(X[:, 0] > -10) & (X[:, 0] < 10)] = 0.5
Y[(X[:, 0] > 10)] = 0

In [ ]:
plt.plot(X, Y, 'b.-')

In [ ]:
# defin the model 
class One_layer_net(torch.nn.Module):    
    def __init__(self, input_size, hidden_neurons, output_size):
        super(One_layer_net, self).__init__()        
        # hidden layer 
        self.linear1 = torch.nn.Linear(input_size, hidden_neurons)
        # output layer 
        self.linear2 = torch.nn.Linear(hidden_neurons, output_size) 
        # defining layers as attributes
        self.layer_in = None
        self.hidden = None
        self.layer_out = None

    def forward(self, x):
        self.layer_in = torch.sigmoid(self.linear1(x))
        self.hidden = self.linear2(self.layer_in)
        self.layer_out = torch.sigmoid(self.hidden)

        return self.layer_out
    

In [ ]:
# instantiate the model 
model = One_layer_net(1, 2, 1)  # 2 represents two neurons in one hidden layer

### Train the model

In [ ]:
# loss function
# def criterion(y_pred, y):
#     # cross entropy loss  
#     out = -1 * torch.mean(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred))
#     return out

criterion = nn.MSELoss()

In [ ]:
# optimization procedure: stochastic gradient descent 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Define the training loop
epochs = 2000
cost = []
total = 0

for epoch in range(epochs):
    total=0
    epoch = epoch + 1
    
    for x, y in zip(X, Y):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()
        # get total loss 
        total+=loss.item() 
    cost.append(total)
    
    if epoch % 500 == 0:
        print(str(epoch)+ " " + "epochs done!") 
        # plot the result of function approximator
        plt.plot(X.numpy(), model(X).detach().numpy())
        plt.plot(X.numpy(), Y.numpy(), 'm')
        plt.xlabel('x')
        plt.show()

In [ ]:
# plot the cost
plt.plot(cost)
plt.xlabel('epochs')
plt.title('loss')